In [ ]:
### 爬取 E 大微博列表

import requests
from datetime import datetime
import urllib.parse
import time

headers = {
    'Cookie' : 'XSRF-TOKEN=IMU7h8hbgd2ZniqIeE85b0ee; SUB=_2A25IcCFUDeRhGeNH7FoR8yjIzT6IHXVrDDycrDV8PUNbmtB-LUrCkW9NSmujoHYKlanbz3k0nt48sn3QFEMKArED; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9Whr96MVub61x9nGvhHXExsu5JpX5KzhUgL.Fo-4S0n7e0qXSoz2dJLoI7H0qrUud5tt; WBPSESS=C_z6TpWRc-Q0Bi2FixSaqetj1-a1jJsz59IX_OvZ58BfvH-s34Cgy4qQiVlUkLlMAb8av8Ll9pdrJ_TBzr4m4qRJqvRlu_VFnGJlaKQhwIAgeMo0-RfRctUn-qF3okKhZ-oNkGf4AFtaDyWFte2CWw=='
}

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def etf_weibo_list(year, month):
    _, last_day = calendar.monthrange(year, month)
    start = datetime(year, month, 1)
    end = start + relativedelta(months=1)

    url = 'https://weibo.com/ajax/statuses/searchProfile'
    params = {
        'uid': '5687069307',
        'page': 1,
        'starttime': start.timestamp(),
        'endtime': end.timestamp(),
        'hasori': 1,
        'hasret':1,
        'hastext': 1,
        'haspic': 1,
        'hasvideo': 1,
        'hasmusic': 1
    }

    with open(f'C:\\Users\\wayne\\Desktop\\{year}-{month}.jsonl', 'a', encoding='utf-8') as f:
        for page in range(1, 200):
            params['page'] = page
            response = requests.get(url + '?' + urllib.parse.urlencode(params), headers=headers)
            f.write(response.text)
            f.write('\n')
            if len(response.json().get('data', {}).get('list')) == 0:
                break
            time.sleep(1)
            print(f'{year}-{month} {page} complete')


year = 2015

for i in range(0, 12):
    month = 12 - i
    etf_weibo_list(year, month)
    time.sleep(5)


In [ ]:
### 爬取 E 大部分长文本

import json
import os


def merge_to_one(year):
    count = 0
    with open(f'C:\\Users\\wayne\\Desktop\\{year}-long-text.jsonl', 'a', encoding='utf-8') as af:
        with open(f'C:\\Users\\wayne\\Desktop\\5687069307\\{year}.jsonl', 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                response_body = json.loads(line)
                for weibo in response_body['data']['list']:
                    if weibo['isLongText']:
                        count = count + 1
                        mblogid = weibo['mblogid']
                        response = requests.get(f'https://weibo.com/ajax/statuses/longtext?id={mblogid}', headers=headers)
                        body = response.json()
                        body['mblogid'] = mblogid
                        af.write(json.dumps(body, ensure_ascii=False))
                        af.write('\n')
                        if count % 30 == 0:
                            print(f'{year} {mblogid}')
                            time.sleep(5)
                        time.sleep(1)

for year in range(2017, 2023):
    merge_to_one(year)

In [9]:
### 保存到 sqlite db 中

import sqlite3
import json
from datetime import datetime

conn = sqlite3.connect(f'C:\\Users\\wayne\\Desktop\\etf-5687069307.db')
cursor = conn.cursor()
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS weibo (
#         mblogid TEXT PRIMARY KEY,
#         text TEXT,
#         text_raw TEXT,
#         created_at NUM
#     )
# ''')
# conn.commit()

date_format = "%a %b %d %H:%M:%S %z %Y"

# 使用 strptime 将字符串转换为 datetime 对象

def etl(year):
    with open(f'C:\\Users\\wayne\\Desktop\\5687069307\\{year}.jsonl', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            response_body = json.loads(line)
            for weibo in response_body['data']['list']:
                mblogid = weibo['mblogid']
                text = weibo['text']
                text_raw = weibo['text_raw']
                created_at = int(datetime.strptime(weibo['created_at'], date_format).timestamp())
                try:
                    cursor.execute(f'''insert into weibo (mblogid, text, text_raw, created_at) 
                    values (?, ?, ?, ?)''', (mblogid, text, text_raw, created_at))
                    conn.commit()
                except Exception as e:
                    print(weibo)
                    raise e
try:
    etl(2015)
finally:
    conn.close()
